In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.utils import py_random_state
from networkx.generators.random_graphs import _random_subset
from networkx.generators.classic import star_graph
import numpy as np
import random

import scipy.stats as stats
from scipy.optimize import curve_fit


import time
import os

In [2]:
seed = np.random.RandomState()           
time_steps = int(1e5) 
# t0 = time.time()

explorations_per_graph=3

def LinearSearch(lys, element):
    for i in range(len(lys)):
        if lys[i] == element:
            return i
    return -1

#definisco una funzione del grafo per poi fare la media tra grafi differenti

def time_nodes (G, explorations_per_graph):
    initial_node = _random_subset(G.nodes, 1, seed).pop()
       
    #dinamica di esplorazione: semplicemente riempio una lista dei nuovi nodi in cui va il walker. Linear search 
    #mi guarda se il nodo è gia presente e non lo aggiunge. Exploration fa un evoluzione temporale restituendo il nuovo nodo
    
    def exploration(initial_node, G, explored_nodes):
        if LinearSearch(explored_nodes, initial_node) < 0:
               explored_nodes += [initial_node]
        neighbours = list(G[initial_node])
        new_node = _random_subset(neighbours, 1, seed).pop()
        return new_node

    exploration_time = []
    explored_nodes = []

    #ripeto l'esplorazione di "time_steps" passi temporali "exploration_per_graph" volte per fare una media - i tempi 
    #di esplorazione variano molto di volta in volta. Semplicemente ogni iterazione mi cambia il nodo iniziale con quello
    #nuovo chiamando la funzione esplorazione. Quando tutti i nodi sono esplorati salvo il tempo impiegato e svuoto il vettore
    #Infine faccio una media dei tempi.
    
    for i in range(explorations_per_graph):
        t = 0
        while True:
            initial_node = exploration(initial_node, G, explored_nodes)
            if len(explored_nodes) == len(G.nodes):
                exploration_time += [t]
                explored_nodes.clear()
                break
            t += 1

    mean_exploration_time = sum(exploration_time)/explorations_per_graph
    print("mean_exploration_time: ", mean_exploration_time, "for", len(G.nodes) ,"nodes")

    return mean_exploration_time
    
#print(exploration_time)
#print(explored_nodes) 
#print(len(explored_nodes),"Explored nodes")
     

# print(f"done in {int((time.time()-t0)/60)} minutes and {((time.time()-t0)%60)} seconds")

In [ ]:
t0 = time.time()

Times = []
N = [200+500*i for i in range(30)]

for i in range(len(N)):
    G = nx.complete_graph(N[i])
    Times += [time_nodes(G,explorations_per_graph)]
    
print(f"done in {int((time.time()-t0)/60)} minutes and {((time.time()-t0)%60)} seconds")

mean_exploration_time:  1417.6666666666667 for 200 nodes
mean_exploration_time:  4713.333333333333 for 700 nodes
mean_exploration_time:  9483.666666666666 for 1200 nodes
mean_exploration_time:  14951.0 for 1700 nodes
mean_exploration_time:  21627.0 for 2200 nodes
mean_exploration_time:  24176.666666666668 for 2700 nodes
mean_exploration_time:  25630.333333333332 for 3200 nodes
mean_exploration_time:  38385.333333333336 for 3700 nodes
mean_exploration_time:  38283.0 for 4200 nodes
mean_exploration_time:  43710.666666666664 for 4700 nodes
mean_exploration_time:  40123.0 for 5200 nodes
mean_exploration_time:  57267.333333333336 for 5700 nodes
mean_exploration_time:  63444.666666666664 for 6200 nodes
mean_exploration_time:  64197.333333333336 for 6700 nodes
mean_exploration_time:  75742.33333333333 for 7200 nodes
mean_exploration_time:  79316.33333333333 for 7700 nodes
mean_exploration_time:  73398.66666666667 for 8200 nodes
mean_exploration_time:  78400.0 for 8700 nodes
mean_exploration_t

In [ ]:
fig, ax = plt.subplots()
    
ax.plot(N, Times, '.', label = 'exploration_time')
ax.set_xlabel("Number of nodes")
ax.set_ylabel("Exploration time")

plt.show()

linear_fit=np.polyfit(N,Times,1)
def log_law(x, a):
    return a*x*np.log(x) 

popt, pcov = curve_fit(log_law, N, Times)

print(linear_fit)

fit = [linear_fit[0]*n+linear_fit[1] for n in N]
fit_log = log_law(np.array(N), *popt)

fig, ax = plt.subplots()
    
ax.plot(N, Times, '.', label = f"exploration time")
ax.plot(N, fit, '-', label = f"fit: {round(linear_fit[1],4)} + x • {round(linear_fit[0],3)}")
ax.plot(N, fit_log, '-', label = f"fit: {round(popt[0],4)} • N • log(N)")

ax.set_xlabel("Number of nodes")
ax.set_ylabel(f"Exploration time")

ax.legend()
plt.show()